https://github.com/keras-team/keras-tuner

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import kerastuner as kt
from kerastuner.applications import HyperResNet, HyperXception
from kerastuner.tuners import Hyperband, BayesianOptimization, RandomSearch

In [2]:
data = tfds.load('cifar10')
train_ds, test_ds = data['train'], data['test']
def standardize_record(record):
  return tf.cast(record['image'], tf.float32) / 255., record['label']
train_ds = train_ds.map(standardize_record).cache().batch(64).shuffle(10000)
test_ds = test_ds.map(standardize_record).cache().batch(64)

In [ ]:
hypermodel = HyperResNet(input_shape=(32,32,3),classes=10)
tuner1 = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    directory='hyperband_test',
    project_name='hello_world')
tuner1.search(train_ds,
             validation_data=test_ds,
             epochs=10,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

INFO:tensorflow:Reloading Oracle from existing project hyperband_test\hello_world\oracle.json
Epoch 1/2
782/782 [==============================]79.3677 - accuracy: 0.3438 - 19s 10s/step - loss: 175.0682 - accuracy: 0.2344 - 19s 6s/step - loss: 167.8913 - accuracy: 0.1979  - 19s 5s/step - loss: 178.3300 - accuracy: 0.1484 - 19s 4s/step - loss: 217.0776 - accuracy: 0.1187 - 20s 3s/step - loss: 287.4046 - accuracy: 0.0990 - 20s 3s/step - loss: 414.4098 - accuracy: 0.0848 - 20s 2s/step - loss: 556.0162 - accuracy: 0.0742

INFO:tensorflow:Reloading Oracle from existing project hyperband_test\hello_world\oracle.json
c:\users\kuki\pycharmprojects\tensorflow-tutorial-kyu\venv\lib\site-packages\tensorflow_core\python\keras\callbacks.py:1261: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):
c:\users\kuki\pycharmprojects\tensorflow-tutorial-kyu\venv\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


In [3]:
def build_model(hp):
  inputs = tf.keras.Input(shape=(32, 32, 3))
  x = inputs
  for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
    filters = hp.Int('filters_' + str(i), 32, 256, step=32)
    for _ in range(2):
      x = tf.keras.layers.Convolution2D(
        filters, kernel_size=(3, 3), padding='same')(x)
      x = tf.keras.layers.BatchNormalization()(x)
      x = tf.keras.layers.ReLU()(x)
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
      x = tf.keras.layers.MaxPool2D()(x)
    else:
      x = tf.keras.layers.AvgPool2D()(x)
  x = tf.keras.layers.GlobalAvgPool2D()(x)
  x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
  x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
  outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
  return model

In [4]:
tuner2 = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2)
tuner2.search(train_ds,
             validation_data=test_ds,
             epochs=10,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

Epoch 1/2
782/782 [==============================]443 - accuracy: 0.0938 - 4s 2s/step - loss: 2.3935 - accuracy: 0.0859 - 4s 1s/step - loss: 2.3753 - accuracy: 0.0729 - 4s 1s/step - loss: 2.3385 - accuracy: 0.0898 - 4s 807ms/step - loss: 2.3023 - accuracy: 0.1156 - 4s 676ms/step - loss: 2.2983 - accuracy: 0.1146 - 4s 582ms/step - loss: 2.2931 - accuracy: 0.1228 - 4s 512ms/step - loss: 2.2718 - accuracy: 0.1348 - 4s 457ms/step - loss: 2.2531 - accuracy: 0.1424

KeyboardInterrupt: 

In [5]:
best_model = tuner2.get_best_models(1)[0]
best_hyperparameters = tuner2.get_best_hyperparameters(1)[0]